In [166]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import h5py
from glob import glob
import os
from random import sample
from tqdm.notebook import tqdm
import re
# fix random seed
np.random.seed(0)

In [210]:
# configuration
# define base path
PREFIX = "/gpfs/exfel/exp/MID/202102/p002841/scratch/proc/"

# specify runs
RUNS = [224]

# all energies in keV
ENERGY_LOW = 6.5
ENERGY_HIGH = 9.5
ENERGY_STEP = 0.25

# Blank out inner tiles, select tiles in general
TILES = [0,1,2,5,6,7,9,10,11,12,13,14]

# how many samples per tile?!
SAMPLES_PER_TILE = 10

In [211]:
# functions

# are runs valid
def is_run_valid(run):
    assert os.path.isdir(PREFIX + "r" + get_run_string(run))

# get files from run
def get_proc_files(run, tile):
    assert (tile > -1 and tile < 16)
    files = glob(PREFIX + "r" + get_run_string(run) + "/" + "CORR-R" + get_run_string(run) + "-AGIPD" + get_tile_string(tile) + "-S*.h5")
    return files
    
# photon number per file
def get_photon_number_per_file(file):
    agipd_tile = get_agipd_tile_from_file_name(file)
    f = h5py.File(file, 'r')
    if "/INSTRUMENT/MID_DET_AGIPD1M-1/DET/" + str(agipd_tile) + "CH0:xtdf/image/data" not in f:
        return None
    data = f["/INSTRUMENT/MID_DET_AGIPD1M-1/DET/" + str(agipd_tile) + "CH0:xtdf/image/data"]
    number_of_images = data.shape[0]
    samples = sample(range(number_of_images), SAMPLES_PER_TILE)
    photon_numbers = np.zeros(len(samples), dtype=np.int32)
    i = 0
    for sampleo in samples:
        image_data = np.array(data[sampleo]).ravel()
        photon_numbers[i] = get_photon_number_per_image(image_data)
        i += 1
    return rms(photon_numbers)
        
def get_photon_number_per_image(image_data):
    photons = np.histogram(image_data, bins=np.arange(ENERGY_LOW, ENERGY_HIGH, ENERGY_STEP))[0]
    return np.sum(photons)
    
# aux functions
def get_run_string(run):
    return str(run).zfill(4)

def get_tile_string(tile):
    assert (tile > -1 and tile < 16)
    return str(tile).zfill(2)

def rms(data):
    return np.sqrt(np.mean(data**2))

def get_all_files_from_runs(runs):
    all_files_from_run = []
    for run in RUNS:
        is_run_valid(run)
        for i in TILES:
            for element in get_proc_files(run, i):
                all_files_from_run.append(element)
    return all_files_from_run

def get_agipd_tile_from_file_name(filename):
    try:
        found = re.search('AGIPD(.+?)-S00', filename).group(1)
        return int(found)
    except AttributeError:
        print("Could not extract AGIPD tile number from file name")

In [212]:
all_files = get_all_files_from_runs(RUNS)
photon_numbers = []
for i in tqdm(range(len(all_files))):
    number_per_file = get_photon_number_per_file(all_files[i])
    if (number_per_file is None):
        continue
    photon_numbers.append(number_per_file)
    tqdm.write(str(np.mean(photon_numbers)*len(TILES)))

mean = np.mean(photon_numbers)*len(TILES)

mean

  0%|          | 0/144 [00:00<?, ?it/s]

16.540858502508264
25.66008134668146
24.69618728219175
21.808475806674807
22.125241930848148
22.68034229615941
22.60128006777991
22.28610013890965
21.54832076486262
20.76169919849532
22.036026925073443
22.882972920983736
23.650694500162675
23.17354223218514
23.699388073593294
22.888996712243653
22.540853537904447
22.462371687687018
21.845042982650806
21.809199845217805
21.470520731478818
21.625666590229475
21.404587977719856
21.685334085270725
22.132454859872293
23.080614782765014
23.490688262285513
24.179037654164357
24.79650625230353
24.7182875212482
24.930348296207058
25.244578392234082
25.707368842127654
26.01009328794743
26.06367576808303
26.156181355452894
26.152375915208612
26.328980587184958
26.159468573928578
26.03368636543013
26.28646967679641
26.348692469946528
26.40802117969432
26.11879581427376
26.25887178743779
26.028159047982328
26.120280462655387
25.882294170864633
25.885010209549325
26.2293076851696
26.650283153978908
26.632722568468406
26.68017872233841
26.65750585938

26.001970800804934